# 임베딩 기반 Evaluator(embedding_distance)

답변과 정답답변 사이의 거리를 측정하는 평가자 생성

distance니까 거리가 가까울수록 유사하다. 즉 0에 가까워야 좋은거다. 

하지만 정답이아닌 할루시네이션을 주더라도 점수가 낮게주고, 간결한 정답인경우에는 점수가 높고 답변의 말투에도 점수가 높게나온다.

즉, 언제사용하면 원하는 reference의 비슷한 형태로 나왔으면 좋겠다고 평가할 때 사용한다. 관련성 평가에는 적합하지 않을 수 있다.

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("CH16-Evaluations")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH16-Evaluations


### RAG 성능 테스트를 위한 함수 정의
테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [3]:
from myrag import PDFRAG
from langchain_openai import ChatOpenAI

rag = PDFRAG(
    "data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatOpenAI(model="gpt-4o-mini", temperature=0)
)

retriever = rag.create_retriever()

chain = rag.create_chain(retriever)

chain.invoke("삼성전자가 자체 개발한 생성형 ai의 이름은 무엇인가요?")

"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."

In [4]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs:dict):
    return {"answer": chain.invoke(inputs["question"])}

### 임베딩 거리기반 Evaluator

In [10]:
from langsmith.evaluation import LangChainStringEvaluator
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
import os

os.environ["TOKENIZERS_PARALLELISM"] = "true"

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [11]:
# huggingface embedding
model_name = "BAAI/bge-m3"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    #model_kwargs={"device":"cuda"},
    model_kwargs={"device": "cpu"}
)

hf_embedding_evaluator = LangChainStringEvaluator(
    "embedding_distance",
    config={
        "embeddings": hf_embeddings,
        "distance_metric": "cosine"
    }
)

In [12]:
# openai embedding
openai_embedding_evaluator = LangChainStringEvaluator(
    "embedding_distance",
    config={
        "embeddings": OpenAIEmbeddings(model="text-embedding-3-small"),
        "distance_metric" : "euclidean"
    }
)

In [13]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_EVAL_DATASET_teddynote"

experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[
        hf_embedding_evaluator,
        openai_embedding_evaluator
    ],
    experiment_prefix="EMBEDDING-EVAL",
    metadata={
        "variant": "embedding_distance 활용한 평가"
    }
    
)

View the evaluation results for experiment: 'EMBEDDING-EVAL-4085ff96' at:
https://smith.langchain.com/o/2d0ce887-3f7f-59af-8d5e-12c1371ef5d5/datasets/334d943e-3086-4d03-91ae-4b5145224ffc/compare?selectedSessions=ccea0e91-68e1-4c5f-94c0-09676c0334cf




0it [00:00, ?it/s]